In [7]:
#Requires gmaps for google map visualization
import sys
import csv, io
import os
import time
import pandas as pd
import numpy as np
import gmplot
import webbrowser
from gmplot import *
from tkinter import *
import gmaps #gmaps for google map visualization
#Main
if __name__=="__main__":
    
    
    fileName='directory.csv'# Specify Input file

#gui

gui = Tk()
#set the background colour of GUI window 
gui.configure(background="#F0F3F4") 



# set the title of GUI window 
gui.title("Nearest neighbour") 



# set the configuration of GUI window 
gui.geometry("800x500") 
gui.grid_rowconfigure(0, weight=1)
gui.grid_columnconfigure(0, weight=1)

title = Label(gui,text='Nearest Neighbour',font=('Times New Roman',35,'bold'),bg='#F0F3F4')
title.pack()
lab = Label(gui, text="Enter No of Nearest Neighbour")
lab.pack()
n=StringVar()
entry = Entry(gui, textvariable= n)
entry.pack()

def find_k():
    try:
        k=int(n.get())
        if (k<1):
            k=28
    except:
        k=28
    #print(k)
    gui.destroy()
button = Button(gui, text="OK", command = find_k)
button.pack()

# start the GUI 
gui.mainloop()    

minPoints=13
eps=13 
#Method to read the File Containing the Locations
def readTextFile():
	#Give appropriate input file path
    fileContent = open(r"C:\Users\rafee\Downloads\Compressed\locations\directory.csv")
    
    initialIndex=1
    
    lines = fileContent.readlines()
    latitude=[]
    longitude=[]
	
    for i in range(initialIndex,len(lines)):
        [lat,lon]= str(lines[i]).split(",")
        if(float(lat)!=0 and float(lon)!=0):
            latitude.append(lat)
            longitude.append(lon)
    return [latitude,longitude]

#compute Distance

def eucledian_dist(lat1,lon1,lat2, lon2):
    return (abs(float(lat2)-float(lat1))**2+abs(float(lon2)-float(lon1))**2)**0.5

#paiwise distance 
def computeDistanceMatrix(lat,longi):
    distanceMatrix=[[0 for i in range(len(lat))] for j in range(len(lat))]
    for i in range(len(lat)):
        for j in range(len(longi)):
            distanceMatrix[i][j]=((i,j),eucledian_dist(lat[i],longi[i],lat[j],longi[j]))
    return distanceMatrix

#K-Nearest Neighbors
def findKNNList(distanceMatrix, k):
    count=len(distanceMatrix)
    
    global similarityMatrix
    count=len(distanceMatrix)
    similarityMatrix = [[0 for i in range(k)] for j in range(count)]
    for i in range(count):
        matrixsorted = sorted(distanceMatrix[i], key=lambda x: x[1])
        #print matrixsorted
        for j in range(k):
            similarityMatrix[i][j] = int(matrixsorted[j+1][0][1])
    return similarityMatrix
#---------end KNN---------------------------------
   
def f(k1,k2,i):
    if i==k1:
        return k2
    else:
        return k1

# Step 1:  K nearest neighbors-----------------------------------------
[lat,lon]= readTextFile()
x= zip(lat,lon,lat)
distanceMatrix=computeDistanceMatrix(lat,lon)#Compute Upper Traingular PairWise Distance Matrix
similarityMatrix=findKNNList(distanceMatrix,k)
count=len(lat)

#Step 2: Construct SNN ---------------------------------------
def countIntersection(listi,listj):
    intersection=0
    for i in listi:
        if i in listj:
            intersection=intersection+1
    return intersection

def sharedNearest(count,k):
    Snngraph= [[0 for i in range(count)] for j in range(count)]
    for i in range(0,count-1):
        nextIndex=i+1
        for j in range(nextIndex,count):
            if j in similarityMatrix[i] and i in similarityMatrix[j]:
                count1=countIntersection(similarityMatrix[i],similarityMatrix[j])
                Snngraph[i][j]=count1
                Snngraph[j][i]=count1
    return Snngraph
sharedNearestN= sharedNearest(count,k)# count of shared neighbors between points


#Step3: SNN density --------------------------------------------------
def density(x,eps):
    numbPoints=0
    for i in range(0,count):
        if x[i] >= eps:
            numbPoints=numbPoints+1
    return numbPoints
snnDensity1=[None for i in range(len(sharedNearestN))]
for i in range(len(sharedNearestN)):
        snnDensity1[i]=density(sharedNearestN[i],eps)


#Step4:core points-------------------------------------------------------------
def coreornot(x,minPoints):
    if x >= minPoints:
        return True
    else:
        return False
def core(x,y):
    if x >= minPoints:
        return y
    else:
        return None
		
coreOrNot=[None for i in range(len(snnDensity1))]
for i in range(len(snnDensity1)):
    coreOrNot[i]=coreornot(snnDensity1[i],minPoints)
corePointsList1=[]
snnDensity2=list(zip(snnDensity1, [i for i in range(len(snnDensity1))]))
for i in range(len(snnDensity2)):
    corePointsList1.append(core(snnDensity2[i][0],snnDensity2[i][1]))
corePointsList=[ x for x in corePointsList1 if x!=None]



#Step5: cluster core points------------------------------------------------
#If two core points are within Eps radius they belong to the same cluster
def findCoreNeighbors(p,corePts,sharednearestNeighbors,eps):
    coreNeighbors=[]
    p2=None
	
    for i in range(0,len(corePts)):
        p2=corePts[i]
        #if two core points share more than eps neighbors make the core point core nearest neighbor of other
        if(p!=p2 and sharednearestNeighbors[p][p2]>=eps):
            coreNeighbors.append(p2)
    return coreNeighbors

def expandCluster(labels,neighborCore,corePts,C,sharednearestNeighbors,eps,visited):
    while len(neighborCore)>0:
            p=neighborCore.pop(0)
            if p in visited:
                continue
            labels[p]=C
            visited.append(p)
            neighCore=findCoreNeighbors(p,corePts,sharednearestNeighbors,eps)
            neighborCore.extend(neighCore)
    return labels
	
visited=[]
labels=[0 for i in range(count)]
neighborCore=[]
c=0 

for i in range(0,len(corePointsList)):
    p=corePointsList[i]
    if p in visited:
        continue
    visited.append(p)
    c=c+1
    labels[p]=c
    neighborCore = findCoreNeighbors(p, corePointsList, sharedNearestN, eps)
    labels=  expandCluster(labels, neighborCore, corePointsList, c,sharedNearestN, eps, visited)


#Step 5: Compute the final cluster--------------------------------------------------------
#All points that are not within a radius of Eps of a core point are discarded (noise)

for i in range(count):
    notNoise=False
    maxSim=sys.maxsize
    bestCore=-1
    sim=None
    if(coreOrNot[i]):#core Point
        continue
    for j in range(len(corePointsList)):
        p=corePointsList[j]
        
        sim=sharedNearestN[i][p]
        
        if(sim>=eps):
            notNoise=True
         
        else:
            labels[i]=0
            break
        
        if(sim>maxSim):
            maxSim=sim
            bestCore=p
        
    if(notNoise):
        labels[i]=labels[bestCore]         
print (labels) #Labels consist of the cluster index


#visualize all the results---------------------------------------------------------------------------
for i in range(len(lat)):
    lat[i]=float(lat[i])
    lon[i]=float(lon[i])
    labels[i]=int(labels[i])
	
zipppedLoc=list(zip(lat,lon,labels))
zipppedLocList=[list(i) for i in zipppedLoc]
name1=str(fileName.replace('.csv',''))+'k'+str(k)+'eps'+str(eps)+'minpoints'+str(minPoints)
name=name1+'.txt'
#Give appropriate path here
filePaths=r'C:\Users\rafee\Downloads\Compressed\locations\results'+name
filePaths1=r'C:\Users\rafee\Downloads\Compressed\locations\results2'+name1
#Write labels to file
outfile=open(filePaths,'w')
for i in range(len(labels)):
    outfile.write(str(labels[i]))
    outfile.write('\n')
	
numberOfClusters=max(labels)

print (numberOfClusters)
clusterLat = [0 for x in range(numberOfClusters+1)]
clusterLong= [0 for x in range(numberOfClusters+1)]
cluster=[0 for x in range(numberOfClusters+1)]
longMean=float(sum(lon)) / len(lon)
latMean=float(sum(lat)) / len(lat)
zipppedLoc=list(zip(lat,lon,labels))
zipppedLocList=[list(i) for i in zipppedLoc]

for i in range(numberOfClusters+1):
    clusterLat[i]=[t[0] for t in zipppedLoc if t[2] == i]
    clusterLong[i]=[t[1] for t in zipppedLoc if t[2] == i]
    cluster[i]=[[t[0],t[1]] for t in zipppedLoc if t[2] == i]




#Plot selected clustrs in Google Map
interesting=[1,2,3]
colors=['#FF0000','#00FF00','#0000FF','#000000','#FF00FF','#FFF000','#FFFFFF','#FFF0F0','#F0FFF0','#F00FFF','#FFF0FB']
gmap = gmplot.GoogleMapPlotter(longMean,latMean, 11)
j=0
pth=filePaths1+'.html'
nameabs=name1+'.html'
url = 'file://'+pth
for i in range(numberOfClusters):
    gmap.scatter(clusterLong[i],clusterLat[i], colors[j], size=1000, marker=False)
    j=j+1
    if j==len(colors):
        j=0
    #Save the visualization in html file
    gmap.draw(pth)
    gmap.draw(nameabs)
	
new = 0
#Open the saved html file in browser
webbrowser.open(url,new=new)



[0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 2, 0, 2, 2, 3, 2, 2, 3, 2, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 3, 2, 2, 2, 3, 2, 3, 3, 2, 2, 2, 2, 3, 2, 2, 0, 2, 0, 3, 2, 3, 2, 2, 2, 2, 0, 0, 2, 2, 2, 3, 2, 2, 3, 3, 3, 2, 3, 2, 3, 3, 3, 2, 2, 3, 0, 2, 2, 3, 0, 0, 2, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 0, 5, 5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 0, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 8, 8, 8, 8, 8, 0, 5, 0, 0, 0, 7, 7, 0, 0, 4, 4, 4, 4, 4, 0, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 

True